In [1]:
import pandas as pd
ev_data = pd.read_csv("final_dataset2.csv")
print(ev_data.columns)

Index(['Unnamed: 0', 'State Name', 'Two Wheeler', 'Three Wheeler',
       'Four Wheeler', 'Goods Vehicles', 'Public Service Vehicle',
       'Special Category Vehicles', 'Ambulance/Hearses',
       'Construction Equipment Vehicle', 'Other', 'Grand Total',
       'total-charging-stations'],
      dtype='object')


In [2]:

ev_data = pd.read_csv("EV_Dataset2.csv")
print(ev_data.columns)

Index(['Year', 'Month_Name', 'Date', 'State', 'Vehicle_Class',
       'Vehicle_Category', 'Vehicle_Type', 'EV_Sales_Quantity'],
      dtype='object')


In [10]:
import pandas as pd
ev_df = pd.read_csv(r"C:\Users\VICTUS\OneDrive\Desktop\projectev\EV_Dataset2.csv")
print(ev_df.describe())
print(ev_df.head())


               Year  EV_Sales_Quantity
count  95985.000000       95985.000000
mean    2018.574590          35.949669
std        2.863236         421.204927
min     2014.000000           0.000000
25%     2016.000000           0.000000
50%     2019.000000           0.000000
75%     2021.000000           0.000000
max     2023.000000       20584.000000
   Year Month_Name      Date           State         Vehicle_Class  \
0  2014        jan  1/1/2014  Andhra Pradesh       ADAPTED VEHICLE   
1  2014        jan  1/1/2014  Andhra Pradesh  AGRICULTURAL TRACTOR   
2  2014        jan  1/1/2014  Andhra Pradesh             AMBULANCE   
3  2014        jan  1/1/2014  Andhra Pradesh   ARTICULATED VEHICLE   
4  2014        jan  1/1/2014  Andhra Pradesh                   BUS   

  Vehicle_Category Vehicle_Type  EV_Sales_Quantity  
0           Others       Others                  0  
1           Others       Others                  0  
2           Others       Others                  0  
3           Othe

In [13]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# Load dataset
ev_df = pd.read_csv(r"C:\Users\VICTUS\OneDrive\Desktop\projectev\EV_Dataset2.csv")

# Aggregate EV sales by State and Year
agg_df = ev_df.groupby(['State', 'Year']).agg({'EV_Sales_Quantity':'sum'}).reset_index()
agg_df['EV_Sales_Lakhs'] = agg_df['EV_Sales_Quantity'] / 100000  # scale to Lakhs

# Use numeric features
states = pd.get_dummies(agg_df['State'], drop_first=True)
X = pd.concat([agg_df[['Year']], states], axis=1)
y = agg_df['EV_Sales_Lakhs']

# Train Linear Regression
model = LinearRegression()
model.fit(X, y)

# Predict future years for sample states
sample_inputs = [
    {"Year":2025, "State":"Maharashtra"},
    {"Year":2028, "State":"Karnataka"},
    {"Year":2030, "State":"Delhi"},
]

print("\n----- Sample Predictions -----")
for inp in sample_inputs:
    df = pd.DataFrame({"Year":[inp["Year"]]})
    for s in states.columns:
        df[s] = 1 if s == f"{inp['State']}" else 0
    pred = model.predict(df)[0]
    pred = max(min(pred, 15), 0.5)  # bound 0.5–15 Lakhs
    print(f"Prediction {inp['Year']}, {inp['State']}: {pred:.2f} Lakhs")



----- Sample Predictions -----
Prediction 2025, Maharashtra: 0.64 Lakhs
Prediction 2028, Karnataka: 0.68 Lakhs
Prediction 2030, Delhi: 0.72 Lakhs


In [ ]:

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import matplotlib.pyplot as plt
import numpy as np

# -----------------------------
# Load dataset
# -----------------------------
ev_df = pd.read_csv(r"C:\Users\VICTUS\OneDrive\Desktop\projectev\EV_Dataset2.csv")

# Aggregate EV sales by State and Year
agg_df = ev_df.groupby(['State', 'Year']).agg({'EV_Sales_Quantity': 'sum'}).reset_index()
agg_df['EV_Sales_Lakhs'] = agg_df['EV_Sales_Quantity'] / 100000  # scale to Lakhs

# -----------------------------
# Prepare features (Year + State dummies)
# -----------------------------
states = pd.get_dummies(agg_df['State'], drop_first=False)  # Keep all states
X = pd.concat([agg_df[['Year']], states], axis=1)
y = agg_df['EV_Sales_Lakhs']

# -----------------------------
# Train Linear Regression model
# -----------------------------
model = LinearRegression()
model.fit(X, y)

# -----------------------------
# Save model as PKL file
# -----------------------------
with open(r"C:\Users\VICTUS\OneDrive\Desktop\projectev\ev_model9.pkl", "wb") as f:
    pickle.dump({"model": model, "columns": X.columns.tolist()}, f)

print("✅ Model saved as 'ev_model9.pkl' successfully!")

# -----------------------------
# Evaluate model performance
# -----------------------------
y_pred = model.predict(X)
mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y, y_pred)

print("\n----- Model Evaluation -----")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

# -----------------------------
# Predict future EV sales
# -----------------------------
sample_inputs = [
    {"Year": 2025, "State": "Maharashtra"},
    {"Year": 2029, "State": "Karnataka"},
    {"Year": 2034, "State": "Delhi"},
]

future_preds = []

print("\n----- Sample Predictions -----")
for inp in sample_inputs:
    # Build a row with the same columns as training
    df = pd.DataFrame(columns=X.columns)
    df.loc[0] = 0  # initialize with zeros
    df["Year"] = inp["Year"]
    if inp["State"] in states.columns:
        df[inp["State"]] = 1
    pred = model.predict(df)[0]
    future_preds.append({"Year": inp["Year"], "State": inp["State"], "Predicted_Lakhs": pred})
    print(f"Prediction {inp['Year']}, {inp['State']}: {pred:.2f} Lakhs")

# -----------------------------
# Data Visualization - Sales
# -----------------------------
plt.figure(figsize=(12, 6))

# Plot historical sales for each state
for state in agg_df['State'].unique():
    state_data = agg_df[agg_df['State'] == state]
    plt.plot(state_data['Year'], state_data['EV_Sales_Lakhs'], marker='o', label=f'{state} (Historical)')

# Plot future predictions
for pred in future_preds:
    plt.scatter(pred['Year'], pred['Predicted_Lakhs'], color='red', s=100, label=f"{pred['State']} (Predicted)")
    plt.text(pred['Year'], pred['Predicted_Lakhs'] + 0.1, f"{pred['Predicted_Lakhs']:.2f}", ha='center')

plt.title("EV Sales: Historical vs Future Predictions")
plt.xlabel("Year")
plt.ylabel("EV Sales (Lakhs)")
plt.grid(True)
plt.legend()
plt.show()

# -----------------------------
# Data Visualization - Residuals
# -----------------------------
residuals = y - y_pred

plt.figure(figsize=(10, 5))
plt.scatter(y_pred, residuals, alpha=0.6, color="purple")
plt.axhline(y=0, color="red", linestyle="--", linewidth=1)
plt.title("Residuals Plot (Actual - Predicted)")
plt.xlabel("Predicted EV Sales (Lakhs)")
plt.ylabel("Residuals (Lakhs)")
plt.grid(True)
plt.show()


✅ Model saved as 'ev_model9.pkl' successfully!

----- Model Evaluation -----
Mean Absolute Error (MAE): 0.1284
Mean Squared Error (MSE): 0.0453
Root Mean Squared Error (RMSE): 0.2127
R² Score: 0.4275

----- Sample Predictions -----
Prediction 2025, Maharashtra: 0.64 Lakhs
Prediction 2029, Karnataka: 0.72 Lakhs
Prediction 2034, Delhi: 0.88 Lakhs


C:\Users\VICTUS\AppData\Local\Temp\ipykernel_16444\3509944197.py:175: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\VICTUS\AppData\Local\Temp\ipykernel_16444\3509944197.py:189: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [27]:
import pickle

with open("ev_model9.pkl", "rb") as f:
    ev_data = pickle.load(f)

# Show keys
print(ev_data.keys())

# Show first 20 feature columns
print(ev_data["columns"][:20])


dict_keys(['model', 'columns'])
['Year', 'Andaman & Nicobar Island', 'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chandigarh', 'Chhattisgarh', 'DNH and DD', 'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jammu and Kashmir', 'Jharkhand', 'Karnataka', 'Kerala', 'Ladakh', 'Madhya Pradesh']


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pickle

# -----------------------------
# Load Charging Station Dataset
# -----------------------------
charging_df = pd.read_csv(r"C:\Users\VICTUS\OneDrive\Desktop\projectev\final_dataset2.csv")

# Target column
target_col = "total-charging-stations"

# Clean dataset (drop NA)
charging_df = charging_df.dropna(subset=[target_col, "State Name"])

# Features (all vehicle categories + state)
Xc = pd.get_dummies(
    charging_df.drop(columns=[target_col, "Unnamed: 0"]),  # remove target + index col
    drop_first=False  # keep all columns to avoid constant prediction
)
yc = charging_df[target_col]

# Train-test split
Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.2, random_state=42)

# Train Random Forest
cs_model = RandomForestRegressor(n_estimators=200, random_state=42)
cs_model.fit(Xc_train, yc_train)

# Evaluate
yc_pred = cs_model.predict(Xc_test)
rmse = np.sqrt(mean_squared_error(yc_test, yc_pred))
r2 = r2_score(yc_test, yc_pred)

print(f"Charging Station RMSE: {rmse:.2f}")
print(f"Charging Station R²: {r2:.2f}")

# -----------------------------
# Save model & columns as pickle
# -----------------------------
with open(r"C:\Users\VICTUS\OneDrive\Desktop\projectev\cs_model.pkl", "wb") as f:
    pickle.dump({"model": cs_model, "columns": Xc.columns.tolist()}, f)

print("✅ Charging Station model saved as cs_model.pkl")

# -----------------------------
# Function for prediction
# -----------------------------
def predict_charging_stations(state, two_wheeler, three_wheeler, four_wheeler,
                              goods_vehicle, public_service, special_category,
                              ambulance, construction, other, grand_total):
    # Load model
    with open(r"C:\Users\VICTUS\OneDrive\Desktop\projectev\cs_model.pkl", "rb") as f:
        data = pickle.load(f)
        model = data["model"]
        columns = data["columns"]

    # Prepare input
    input_df = pd.DataFrame([{
        "State Name": state,
        "Two Wheeler": two_wheeler,
        "Three Wheeler": three_wheeler,
        "Four Wheeler": four_wheeler,
        "Goods Vehicles": goods_vehicle,
        "Public Service Vehicle": public_service,
        "Special Category Vehicles": special_category,
        "Ambulance/Hearses": ambulance,
        "Construction Equipment Vehicle": construction,
        "Other": other,
        "Grand Total": grand_total
    }])
    input_encoded = pd.get_dummies(input_df, drop_first=False)
    input_encoded = input_encoded.reindex(columns=columns, fill_value=0)

    # Predict
    prediction = model.predict(input_encoded)[0]
    return max(int(prediction), 1)

# Sample prediction
sample_prediction = predict_charging_stations(
    "Maharashtra", 500000, 100000, 200000,
    50000, 30000, 20000, 1000, 5000, 2000, 900000
)
print("Sample Prediction:", sample_prediction, "stations")

Charging Station RMSE: 14.50
Charging Station R²: 0.78
✅ Charging Station model saved as cs_model.pkl
Sample Prediction: 189 stations
